In [1]:
term = 25
target = "unsold_margin"
target = "purchase_margin"

target_col = "diff_unsold_margin_volume_ratio"
target_col = "diff_purchase_margin_volume_ratio"
desc = False

In [2]:
# 読み込みファイルパスの設定とimportしたいmoduleパス(pythonパス)の設定
from pathlib import Path
import os, sys

CURRENT_DIR = Path(os.getcwd())
PJ_DIR = CURRENT_DIR.parent.parent
LIB_DIR = PJ_DIR / "lib"
DATA_DIR = PJ_DIR / "data" 

sys.path.append(str(LIB_DIR))

# notebook内で利用するmoduleのimport
from lib_dataprocess import CreditbalancePl, PricelistPl
import polars as pl
from datetime import date

In [3]:
CPL = CreditbalancePl()
CPL.with_columns_margin_volume_ratio()

In [4]:
# margin_volume_ratioの週の差分を取ってみる
df = CPL.df
ori_cols = df.columns
scol = "unsold_margin_volume_ratio"
pcol = "purchase_margin_volume_ratio"
s_scol = f'shifted_{scol}'
s_pcol = f'shifted_{pcol}'
d_scol = f'diff_{scol}'
d_pcol = f'diff_{pcol}'
df = df.with_columns([
    pl.col(scol).shift().alias(s_scol),
    pl.col(pcol).shift().alias(s_pcol)
])
df = df.with_columns([
    (pl.col(scol) - pl.col(s_scol)).alias(d_scol),
    (pl.col(pcol) - pl.col(s_pcol)).alias(d_pcol)
])
df = df.drop_nulls()
added_cols = [d_scol, d_pcol]
df = df.select(["code", "date", f'volume_ma{term}']+[target]+added_cols)
diff_df = df


In [5]:
# 急増、急減を調べる
df = diff_df



df = df.sort(by=[target_col], descending=[desc])
df

code,date,volume_ma25,purchase_margin,diff_unsold_margin_volume_ratio,diff_purchase_margin_volume_ratio
i64,date,f64,i64,f64,f64
3322,2022-10-07,1968.0,629300,0.0,-1301.88
3322,2021-11-05,1396.0,604000,0.0,-901.41
7864,2009-02-13,74808.0,45600,2.39,-791.38
1739,2013-10-04,232.92,45400,0.0,-652.08
3322,2023-07-14,512.0,632100,0.0,-626.61
…,…,…,…,…,…
3439,2020-04-24,200.0,155900,0.0,732.46
1739,2013-09-27,56.08,47500,0.0,824.58
3322,2022-11-25,504.0,630300,0.0,954.55


In [9]:
df = CPL.df
df = df.filter(pl.col("code")==3322).filter(pl.col("date")>=date(2022, 8, 1))
df

code,date,unsold_margin,purchase_margin,volume_ma25,unsold_margin_volume_ratio,purchase_margin_volume_ratio
i64,date,i64,i64,f64,f64,f64
3322,2022-08-12,0,629400,1960.0,0.0,321.12
3322,2022-08-19,0,629400,1868.0,0.0,336.94
3322,2022-08-26,0,629500,1708.0,0.0,368.56
3322,2022-09-30,0,629200,388.0,0.0,1621.65
3322,2022-10-07,0,629300,1968.0,0.0,319.77
…,…,…,…,…,…,…
3322,2024-11-08,0,636700,3724.0,0.0,170.97
3322,2024-11-15,400,641300,3900.0,0.1,164.44
3322,2024-11-22,0,602700,12864.0,0.0,46.85


In [7]:
RawPl = PricelistPl(str(DATA_DIR/"raw_pricelist.parquet"))
RawPl.df.filter(pl.col("code")==3322).filter(pl.col("date")>=date(2022,10,3))

code,date,open,high,low,close,volume,split_rate
i64,date,f64,f64,f64,f64,f64,f64
3322,2022-10-03,800.0,800.0,779.0,779.0,20700.0,1.0
3322,2022-10-04,776.0,776.0,754.0,754.0,19300.0,1.0
3322,2022-10-05,739.0,752.0,737.0,751.0,1000.0,1.0
3322,2022-10-06,749.0,755.0,749.0,755.0,300.0,1.0
3322,2022-10-07,743.0,745.0,715.0,745.0,400.0,1.0
…,…,…,…,…,…,…,…
3322,2024-12-10,877.0,878.0,873.0,875.0,4400.0,1.0
3322,2024-12-11,875.0,877.0,871.0,876.0,5400.0,1.0
3322,2024-12-12,877.0,878.0,876.0,877.0,15900.0,1.0
